# Introduction to RoutingZoo

#### Import libraries

In [1]:
import sys
import os
from pathlib import Path

RoutingZoo = str(Path.home() / "Documents/Simulator_human_behaviour")

sys.path.append(RoutingZoo)

import data_analysis as da
import utilities_RZ as URZ


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from RouteRL.environment.environment import TrafficEnvironment
from RouteRL.keychain import Keychain as kc


from RouteRL.utilities import check_device
from RouteRL.utilities import get_params
from RouteRL.utilities import set_seeds

In [2]:
check_device()
set_seeds()
params = get_params("params.json")

[INFO] Running on device: cpu


#### Environment initialization

In [4]:
kwargs = {
        "learning_type" : 'markow',
        "gamma_c" : 0.2,
        "gamma_u" : 0,
        "remember" : 5,
        "greedy" : 0.3,
        "noise_alpha" : 0,
        "noise_taste" : 0.8,
        "noise_random" : 0.2,
        "network":'grid',
        "demand":params[kc.AGENT_GEN][kc.NUM_AGENTS]
        }

In [5]:
env = TrafficEnvironment(params[kc.RUNNER], params[kc.ENVIRONMENT], params[kc.SIMULATOR], params[kc.AGENT_GEN], params[kc.AGENTS], params[kc.PLOTTER],**kwargs)

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: /opt/homebrew/opt/sumo/share/sumo/tools


In [6]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  1000 

Agents are:  [<agent.HumanAgent object at 0x1064bde50>, <agent.HumanAgent object at 0x16b0d4970>, <agent.HumanAgent object at 0x16b025130>, <agent.HumanAgent object at 0x16b0d4280>, <agent.HumanAgent object at 0x1065bf430>, <agent.HumanAgent object at 0x16b0d4af0>, <agent.HumanAgent object at 0x16b025040>, <agent.HumanAgent object at 0x1065bf490>, <agent.HumanAgent object at 0x16b0d4be0>, <agent.HumanAgent object at 0x16b0d46a0>, <agent.HumanAgent object at 0x1064bdb50>, <agent.HumanAgent object at 0x16b0d43d0>, <agent.HumanAgent object at 0x16b025100>, <agent.HumanAgent object at 0x1065bf400>, <agent.HumanAgent object at 0x16b0d4910>, <agent.HumanAgent object at 0x106586220>, <agent.HumanAgent object at 0x106586eb0>, <agent.HumanAgent object at 0x106586e50>, <agent.HumanAgent object at 0x106586d60>, <agent.HumanAgent object at 0x106586280>, <agent.HumanAgent object at 0x106586cd0>, <agent.HumanAgent object at 0x106586a90>, <agent.HumanAgent object at

#### Human learning

In [7]:
num_episodes =  300

env.start()

for episode in range(num_episodes):
    
    env.step()

    URZ_help = URZ.Utilities(env,RoutingZoo,episode,**kwargs)

    URZ_help.data()

    URZ_help.Replace_data()

    env.reset()

env.close()

 Retrying in 1 seconds


In [8]:
program = da.Table_record_creator(num_episodes,URZ_help.model,50,3)
program.table_record()

Run was succesful


,network,model,demand,Bounded,Greedy,link_value,link_std,TT_value,TT_std,entropy_value,entropy_std
0,grid,markow,1000,0.2,0.3,11.0,4.1,62684.0,4444.18,0.73,0.2


In [ ]:
    def act(self, state):

        self.make_noise()

        utilities = list(map(lambda x,y: ((self.beta * x) + y), self.cost,self.stored_noises)) #RK: here to do: self.noise[k]

        if abs(self.old_c_hat[-2] - self.old_c_hat[-1]) > self.gamma_u:

            p = np.random.random()

            if p < self.greedy:

                action = np.random.randint(low=0,high=self.action_space) #RK: First bounded rationality, then greedy.

            else:

                action = np.argmin(utilities)
                self.stored_utilities = utilities
        
        else:

            action = np.argmin(self.stored_utilities)


        return action       


    def learn(self, action, observation):

        reward = self.get_reward(observation)
        self.last_reward = reward

        if abs(self.old_c_hat[-1]-reward) > self.gamma_c:

            if self.type == 'weight':
                c_hat = 0
                if self.cost[action] != reward: #RK: This is quite dangerous assumption. you beeter check this condition somehow else (what if the costs are the same on two paths?)
                    del(self.days[action][len(self.days[action])-1]) #RK: I would try to data structure called queue here https://docs.python.org/3/library/queue.html
                    self.days[action].insert(0,self.cost[action])
                for memory in range(self.remember): #memory

                    c_hat += self.alpha_j[memory] * self.days[action][memory]

            else:

                    c_hat = self.alpha_zero * self.cost[action] + self.alpha_j * reward

            self.cost[action] = c_hat
            self.old_c_hat.append(c_hat)